In [2]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json
import matplotlib.pyplot as plt
import json
import os

We are going to read the energy eigenvalues and dipole matrix elements from .npy files.  We will also still create an instance of the PFHamiltonianGenerator class so we can use its build_pcqed_pf_hamiltonian() method, but it is not really important what details we use to instantiate this class... so we will use LiH in a minimal basis since this is a fast way to instantiate the class!

In [3]:
# # Set the molecule and bond length
# mol = psi4.geometry("""
# 0 1
# O
# H 1 1.0
# """)

# # Optimize the geometry to obtain the new coordinates
# psi4.optimize('scf/cc-pvdz')

# # Get the new optimized coordinates
# new_coordinates = mol.geometry()

# # Print the new coordinates
# print(new_coordinates)

In [8]:
# json_file = os.path.join("/Users/rmandern/code/SCQED-PCQED/", 'to', 'data.json')
# with open(json_file, 'r') as file:
#     data = json.load(file)
    
relative_path = 'data.json'
absolute_path = os.path.abspath(relative_path)
with open(absolute_path, 'r') as file:
    data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rmandern/code/SCQED-PCQED/src/data.json'

In [9]:
# read data from .npy files for formaldehyde casci(8,8) calculations

# !!! Change this to the correct path on your computer!
npy_folder = "/Users/rmandern/code/SCQED-PCQED/LiH_data/"

# these file names should still be good
E_npy_file = npy_folder + "LiH_r_scan_6311g_fci_Energies.npy"
Mu_npy_file = npy_folder + "LiH_r_scan_6311g_fci_Dipoles.npy"

# store energy eigenvalues in E_array
E_array = np.load(E_npy_file)
# store dipole matrix elements in Mu_array
Mu_array = np.load(Mu_npy_file)

# print their shape so we know how many elements we have
print(np.shape(E_array))
print(np.shape(Mu_array))
# print(E_array)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rmandern/code/SCQED-PCQED/LiH_data/LiH_r_scan_6311g_fci_Energies.npy'

In [ ]:
# setup basic arguments to create an instance of the PFHamiltonianGenerator class
mol_str = """
    Li
    H 1 1.4
    symmetry c1
"""


options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}


cavity_free_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 10, 
}

# create the instance of our PFHamiltonianGenerator class
instance = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

In [ ]:
N_R = 21
d_array = np.linspace(0, 20, N_R)
N_l = len(d_array)
N_el = 300
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0, 0, 0.01])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pcqed_22_dse = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0
for d in d_array:
    instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector , E_array[:,ctr], Mu_array[:,:,:,ctr], neglect_DSE=False)      # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pcqed_22_dse[ctr, :] = instance.PCQED_pf_eigs
    ctr += 1

In [ ]:
# N_R = 21
# d_array = np.linspace(0, 20, N_R)
# N_l = len(d_array)
# N_el = 300
# N_ph = 10
# omega = 0.12086
# lambda_vector = np.array([0, 0, 0.01])

# # create an array of zeros to store the PCQED eigenvalues for each value of d
# _pcqed_22_cs = np.zeros((N_l, N_el * N_ph))
# # loop over values of d, build Hamiltonian, capture eigenvalues
# ctr = 0
# for d in d_array:
#     instance.fast_build_pcqed_cs_hamiltonian(N_el, N_ph, omega, lambda_vector , E_array[:,ctr], Mu_array[:,:,:,ctr], neglect_DSE=False)      # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
#     _pcqed_22_cs[ctr, :] = instance.PCQED_pf_eigs
#     ctr += 1

In [ ]:
N_R = 21
d_array = np.linspace(0, 20, N_R)
N_l = len(d_array)
N_el = 300
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0, 0, 0.01])

# create an array of zeros to store the PCQED eigenvalues for each value of d
_pcqed_22_withoutdse = np.zeros((N_l, N_el * N_ph))
# loop over values of d, build Hamiltonian, capture eigenvalues
ctr = 0
for d in d_array:
    instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector ,E_array[:,ctr], Mu_array[:,:,:,ctr], neglect_DSE=True)      # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
    _pcqed_22_withoutdse[ctr, :] = instance.PCQED_pf_eigs
    ctr += 1

In [ ]:
# N_R = 21
# d_array = np.linspace(0, 20, N_R)
# N_l = len(d_array)
# N_el = 300
# N_ph = 10
# omega = 0.12086
# lambda_vector = np.array([0, 0, 0.01])

# # create an array of zeros to store the PCQED eigenvalues for each value of d
# _pcqed_22_withoutdse_cs = np.zeros((N_l, N_el * N_ph))
# # loop over values of d, build Hamiltonian, capture eigenvalues
# ctr = 0
# for d in d_array:
#     instance.fast_build_pcqed_cs_hamiltonian(N_el, N_ph, omega, lambda_vector , E_array, Mu_array, neglect_DSE=True)      # E_array[:,ctr]: 20 energy values for fisrt displacement and so on... 
#     _pcqed_22_withoutdse_cs[ctr, :] = instance.PCQED_pf_eigs
#     ctr += 1

In [ ]:

plt.plot(d_array, _pcqed_22_dse[:,0],'ro-', label="Dse_nocs(300,10)")
# plt.plot(d_array,_pcqed_22_cs[:,0], label='Dse_cs')
plt.plot(d_array,_pcqed_22_withoutdse[:,0], 'bo-', label='no_Dse(300,10)')
# plt.plot(d_array,_pcqed_22_withoutdse_cs[:,0], label='noDse_cs')

# plt.plot(lambda_val[:,2],_pcqed_502[:,2],'ro-', label='2nd')
# plt.plot(lambda_val[:,2],_pcqed_502[:,3],'b', label='3rd')
# plt.plot(lambda_val[:,2],_pcqed_502[:,4],'y', label='4th')

plt.xlabel('Displacement')
plt.ylabel('Energy')
# plt.ylim(-0.0003, -0.0001)
plt.grid()
plt.legend()
plt.savefig('LiH.png')